In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc, html 
from dash.dependencies import Input, Output

In [2]:
# Load Data
assessments = pd.read_pickle('../data/school_based/assessments_clean.pkl')

In [3]:
# Get school levels
available_levels = assessments['school_lvl'].unique()

In [4]:
# Build App
app = JupyterDash(__name__)

In [5]:
# App layout
app.layout = html.Div([
    html.H1("Claytor's Stuff"),
    dcc.Dropdown(
        id='level-dropdown',
        options=[{'label': level, 'value': level} for level in available_levels],
        value='Elementary',
        clearable=False
    ),
    dcc.Graph(id='graph')
])

In [6]:
# Define callback to update the graph based on level selection
@app.callback(
    Output('graph', 'figure'),
    [Input('level-dropdown', 'value')]
)
def update_graph(level):
    # Filter DataFrame for the chosen level
    level_data = assessments[assessments['school_lvl'] == level]

    # Calculate average pct_met_exceeded for each subject_area and year
    average_data = level_data.groupby(['subject_area', 'year'])['pct_met_exceeded'].mean().reset_index()

    # Create a line chart
    fig = px.line(average_data,
                  x='year',
                  y='pct_met_exceeded',
                  color='subject_area',
                  title=f'Average Yearly Proficiency of {level} Schools by Subject Area',
                  height=600,
                  width=800)

    # Update marker settings
    fig.update_traces(mode='markers+lines', marker=dict(size=5))

    # Update legends and axis labels
    fig.update_layout(legend_title='Subject Area',
                      xaxis_title='Year',
                      yaxis_title=f'Average % of Students Who Met or Exceeded {level} Expectations')

    # Add footnotes
    footnote1_text = "* Testing was canceled for the 2019 - 2020 school year due to school closures."
    fig.add_annotation(text=footnote1_text, xref="paper", yref="paper", x=0, y=-0.16, showarrow=False, align='right',
                       font=dict(size=10))
    footnote2_text = "** Science content area not assessed in the 2018 - 2019 school year."
    fig.add_annotation(text=footnote2_text, xref="paper", yref="paper", x=0, y=-0.2, showarrow=False, align='right',
                       font=dict(size=10))

    return fig

In [7]:
# Run the app
if __name__ == '__main__':
    app.run_server(mode="jupyterlab")

Dash is running on http://127.0.0.1:8050/

